In [ ]:
import torch
import clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from tqdm import tqdm

### Hyperparameters

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

### Dataset

In [ ]:
# Load the CUB-200-2011 dataset
def load_cub_dataset(data_dir):
    images = pd.read_csv(os.path.join(data_dir, 'images.txt'), sep=' ', names=['image_id', 'file_path'])
    labels = pd.read_csv(os.path.join(data_dir, 'image_class_labels.txt'), sep=' ', names=['image_id', 'class_id'])
    classes = pd.read_csv(os.path.join(data_dir, 'classes.txt'), sep=' ', names=['class_id', 'class_name'])
    return images, labels, classes
data_dir = 'data'

images, labels, classes = load_cub_dataset(data_dir)

print(images.head())
print(labels.head())
print(classes.head())

print(images.shape)
print(labels.shape)
print(classes.shape)

## Task 1

### CLIP

In [ ]:
clip_model, clip_preprocess = clip.load("ViT-B/32", device=DEVICE, jit=False)
clip_model.eval()

def get_clip_features(img_path):
    img = Image.open(img_path)
    img_input = clip_preprocess(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        img_features = clip_model.encode_image(img_input)
    return img_features

def get_clip_text_features(text):
    text_input = clip.tokenize([text]).to(DEVICE)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_input)
    return text_features

def get_clip_similarity_score(img_features, text_features):
    img_features /= img_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity_score = (100.0 * img_features @ text_features.T).softmax(dim=-1)
    return similarity_score.item()

def recognize_bird_species(img_path):
    img_features = get_clip_features(img_path)
    similarities = []
    for class_name in classes['class_name']:
        text_features = get_clip_text_features(class_name)
        similarity_score = get_clip_similarity_score(img_features, text_features)
        similarities.append((class_name, similarity_score))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[0][0]

In [ ]:
img_id = 1
image_name = images.iloc[img_id]['file_path']
gt_class = labels.iloc[img_id]['class_id']
gt_class_name = classes[classes['class_id'] == gt_class]['class_name'][0]
bird_species = recognize_bird_species(os.path.join(os.path.join(data_dir, 'images'), image_name))
print(bird_species)
pred_class = classes[classes['class_name'] == bird_species]['class_id'][0]
print(f'Predicted class: {pred_class}, class name: {bird_species}')
print(f"GT class: {gt_class}, class name: {gt_class_name}")

#### Run CLIP on all images

In [ ]:
accuracy = 0
with open('clip_predictions.txt', 'w') as f:
    for img_id in tqdm(images['image_id']):
        image_name = images.iloc[img_id]['file_path']
        gt_class = labels.iloc[img_id]['class_id']
        bird_species = recognize_bird_species(os.path.join(os.path.join(data_dir, 'images'), image_name))
        pred_class = classes[classes['class_name'] == bird_species]['class_id'][0]
        f.write(f'{img_id} {pred_class} {int(gt_class == pred_class)}\n')
        accuracy += int(gt_class == pred_class)
    accuracy /= len(images)
    print(f'Accuracy: {accuracy}')